In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [33]:
df[df['press'] == 'Forbes']['url'].iloc[0]

'https://www.forbes.com/sites/sap/2017/11/11/a-grand-vision-for-a-blockchain-powered-overhaul-of-money/'

In [35]:
Forbes = df[df['press'] == 'Forbes']
Forbes

,source,author,title,description,url,urlToImage,publishedAt,content,press
443,"{'id': None, 'name': 'Forbes'}","Adam Winfield, SAP, Adam Winfield, SAP https:/...",SAPVoice: A Grand Vision For A Blockchain-Powe...,"Meet James Zdralek, a man who has devoted much...",https://www.forbes.com/sites/sap/2017/11/11/a-...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-11T13:45:00Z,James Zdralek has devoted much of the past two...,Forbes
1155,"{'id': None, 'name': 'Forbes'}","Dr. Jemma Green, Contributor, Dr. Jemma Green,...",In Blockchain We Trust,Innovation has always brought both excitement ...,https://www.forbes.com/sites/jemmagreen/2017/1...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-14T10:18:00Z,Innovation has always brought both excitement ...,Forbes
1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes
1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes
1422,"{'id': None, 'name': 'Forbes'}","Panos Mourdoukoutas, Contributor, Panos Mourdo...","Bitcoin, Or Bitcoin Cash?",Bitcoin has lost 8% of its value in the last s...,https://www.forbes.com/sites/panosmourdoukouta...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-15T02:43:00Z,(Photo by Dan Kitwood/Getty Images) Bitcoin an...,Forbes
...,...,...,...,...,...,...,...,...,...
744053,"{'id': None, 'name': 'Forbes'}","Susie Violet Ward, Contributor, \n Susie Viole...","SEC Delays Decision On Spot Bitcoin ETFs, Incl...",The U.S. Securities and Exchange Commission ha...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T15:30:27Z,Photo by Celal Gunes/Anadolu Agency\r\nAnadolu...,Forbes
744130,"{'id': None, 'name': 'Forbes'}","Jay Paris, Contributor, \n Jay Paris, Contribu...",Los Angeles Chargers Can’t Cover Their Disappo...,The strange tale of J.C. Jackson’s tenure with...,https://www.forbes.com/sites/jayparis/2023/09/...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T17:51:20Z,Expensive Los Angeles Chargers cornerback J.C....,Forbes
744241,"{'id': None, 'name': 'Forbes'}","Jarrett Wolf, Contributor, \n Jarrett Wolf, Co...",The Sam Bankman-Fried Case Is Not About Crypto...,To truly understand Sam Bankman-Fried's landma...,https://www.forbes.com/sites/jarrettwolf/2023/...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T02:29:02Z,The trial of Sam Bankman-Fried (SBF) is a clas...,Forbes
744331,"{'id': None, 'name': 'Forbes'}","Billy Bambrough, Senior Contributor, \n Billy ...",Elon Musk And X Could Be About To Decide The F...,X chief executive Linda Yaccarino is due to me...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T11:45:40Z,"BitcoinBTC\r\n, ethereum, XRPXRP\r\n and other...",Forbes


In [49]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import re

def get_contents(data):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(data['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")
            content = soup.select(".current-article")

            # If the .at-content-features selector returns an empty list, try the .content selector
            if not content:
                content = soup.select(".middleRightRail .body-container")

            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
            contents.append(cleaned_content)
        except:
            contents.append('')  # Note the lowercase 'append' method name

    return contents


In [55]:
head = Forbes.tail()

In [61]:
# Adding the content to the DataFrame
Forbes['content_full'] = get_contents(Forbes)
Forbes

100%|██████████| 14090/14090 [37:55<00:00,  6.19it/s] 
<ipython-input-61-aea73efea67e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Forbes['content_full'] = get_contents(Forbes)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
443,"{'id': None, 'name': 'Forbes'}","Adam Winfield, SAP, Adam Winfield, SAP https:/...",SAPVoice: A Grand Vision For A Blockchain-Powe...,"Meet James Zdralek, a man who has devoted much...",https://www.forbes.com/sites/sap/2017/11/11/a-...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-11T13:45:00Z,James Zdralek has devoted much of the past two...,Forbes,Share to FacebookShare to TwitterShare to Link...
1155,"{'id': None, 'name': 'Forbes'}","Dr. Jemma Green, Contributor, Dr. Jemma Green,...",In Blockchain We Trust,Innovation has always brought both excitement ...,https://www.forbes.com/sites/jemmagreen/2017/1...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-14T10:18:00Z,Innovation has always brought both excitement ...,Forbes,Share to FacebookShare to TwitterShare to Link...
1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
1422,"{'id': None, 'name': 'Forbes'}","Panos Mourdoukoutas, Contributor, Panos Mourdo...","Bitcoin, Or Bitcoin Cash?",Bitcoin has lost 8% of its value in the last s...,https://www.forbes.com/sites/panosmourdoukouta...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-15T02:43:00Z,(Photo by Dan Kitwood/Getty Images) Bitcoin an...,Forbes,Share to FacebookShare to TwitterShare to Link...
...,...,...,...,...,...,...,...,...,...,...
744053,"{'id': None, 'name': 'Forbes'}","Susie Violet Ward, Contributor, \n Susie Viole...","SEC Delays Decision On Spot Bitcoin ETFs, Incl...",The U.S. Securities and Exchange Commission ha...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T15:30:27Z,Photo by Celal Gunes/Anadolu Agency\r\nAnadolu...,Forbes,Share to FacebookShare to TwitterShare to Link...
744130,"{'id': None, 'name': 'Forbes'}","Jay Paris, Contributor, \n Jay Paris, Contribu...",Los Angeles Chargers Can’t Cover Their Disappo...,The strange tale of J.C. Jackson’s tenure with...,https://www.forbes.com/sites/jayparis/2023/09/...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T17:51:20Z,Expensive Los Angeles Chargers cornerback J.C....,Forbes,Share to FacebookShare to TwitterShare to Link...
744241,"{'id': None, 'name': 'Forbes'}","Jarrett Wolf, Contributor, \n Jarrett Wolf, Co...",The Sam Bankman-Fried Case Is Not About Crypto...,To truly understand Sam Bankman-Fried's landma...,https://www.forbes.com/sites/jarrettwolf/2023/...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T02:29:02Z,The trial of Sam Bankman-Fried (SBF) is a clas...,Forbes,Share to FacebookShare to TwitterShare to Link...
744331,"{'id': None, 'name': 'Forbes'}","Billy Bambrough, Senior Contributor, \n Billy ...",Elon Musk And X Could Be About To Decide The F...,X chief executive Linda Yaccarino is due to me...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T11:45:40Z,"BitcoinBTC\r\n, ethereum, XRPXRP\r\n and other...",Forbes,Share to FacebookShare to TwitterShare to Link...


In [60]:
head['content_full'].iloc[3]

'Share to FacebookShare to TwitterShare to LinkedinBitcoin\nBTC\n, ethereum, XRP\nXRP\n and other major cryptocurrencies are currently braced for volatility after a looming Joe Biden executive order set alarm bells ringing.\n\nSubscribe now to Forbes\' CryptoAsset &amp; Blockchain Advisor and successfully navigate the bitcoin and crypto market rollercoaster ahead of next year\'s historical bitcoin halving!\n\n\nThe bitcoin price is languishing at under half of its all-time high of almost $70,000 per bitcoin it hit in late 2021, weighing on ethereum, XRP and other major cryptocurrencies such as the meme-based dogecoin—which Telsa billionaire and X (Twitter) owner Elon Musk has been revealed to have been "quietly" funding.\n\nNow, X chief executive Linda Yaccarino is due to meet with seven banks that funded Elon Musk\'s takeover of Twitter and its re-brand to X this coming week and will reportedly propose expansion into payments after a leak suggested Musk could turn X into an "updated v

In [62]:
Forbes[Forbes['content_full'] == '']

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
2470,"{'id': None, 'name': 'Forbes'}","Quora, Contributor, Quora, Contributor https:/...",What Is Blockchain Used For Besides Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-17T17:34:00Z,"You get any opportunity, take advantage of it....",Forbes,
2715,"{'id': None, 'name': 'Forbes'}","Peter Tchir, Contributor, Peter Tchir, Contrib...",Will The Real Bitcoin Please Stand Up,The growing Google Trend of searching for ICO ...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-19T14:24:00Z,Life is short and the impact that we all can h...,Forbes,
4088,"{'id': None, 'name': 'Forbes'}","Out of Asia, Contributor, Out of Asia, Contrib...",Putting Singapore's Dollar On Blockchain May P...,Will Singapore become the first country to ful...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-23T03:53:00Z,Success is within someone’s own well-being. It...,Forbes,
...,...,...,...,...,...,...,...,...,...,...
743392,"{'id': None, 'name': 'Forbes'}","Lawrence Wintermeyer, Contributor, \n Lawrence...",The Top 10 Global Crypto & Digital Assets Conf...,Crypto and digital assets conferences have bec...,https://www.forbes.com/sites/lawrencewintermey...,https://imageio.forbes.com/specials-images/ima...,2023-09-28T13:59:05Z,Which crypto and digital asset conference do I...,Forbes,
743505,"{'id': None, 'name': 'Forbes'}","William Skipworth, Forbes Staff, \n William Sk...",Sam Bankman-Fried Could Face 'Very Long Senten...,Sam Bankman-Fried is facing a number of crimin...,https://www.forbes.com/sites/willskipworth/202...,https://imageio.forbes.com/specials-images/ima...,2023-09-28T16:35:17Z,U.S. District Judge Lewis Kaplan told indicted...,Forbes,
743896,"{'id': None, 'name': 'Forbes'}","Sebastien Bardoz, Forbes Councils Member, \n S...",Enhancing Your Marketing Capabilities With 3-D...,The mix of phygital retail (physical + digital...,https://www.forbes.com/sites/forbesbusinessdev...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T11:45:00Z,Sebastien is the VP Sales North America of Wed...,Forbes,
743925,"{'id': None, 'name': 'Forbes'}","Danielle Chemtob, Forbes Staff, \n Danielle Ch...",Forbes Daily: Apple’s Latest Legal Fight With ...,Ron DeSantis' net worth; Recession déjà vu; IR...,https://www.forbes.com/sites/daniellechemtob/2...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T12:15:45Z,This is a published version of the Forbes Dail...,Forbes,


In [64]:
empty_rows = Seeking_Alpha[Seeking_Alpha['content_full'] == '']

empty_rows['url'].iloc[1]

'https://seekingalpha.com/news/3353221-bitcoin-pulls-back-10k-buffett-munger-gates-hammer-away'

In [24]:
coindesk

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
56,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09T23:00:16Z,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...
141,"{'id': None, 'name': 'CoinDesk'}",Alyssa Hertig,Ethereum to ICOs: You're Doing It Wrong,Ethereum devs have harsh words for many ICO la...,https://www.coindesk.com/ethereum-icos-youre-w...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T09:00:52Z,"""Grotesque"" might not be the word you'd think ...",CoinDesk,"""Grotesque"" might not be the word you'd think ..."
167,"{'id': None, 'name': 'CoinDesk'}",Rachel Rose O'leary,Bitcoin Classic Announces Closure In Wake of S...,Proposed bitcoin scaling solution Bitcoin Clas...,https://www.coindesk.com/bitcoin-classic-annou...,https://media.coindesk.com/uploads/2017/11/clo...,2017-11-10T11:22:12Z,Proposed bitcoin scaling solution Bitcoin Clas...,CoinDesk,A group of developers coding under the banner ...
175,"{'id': None, 'name': 'CoinDesk'}",Sujha Sundararajan,US Treasury Secretary: We're Looking 'Carefull...,"The U.S. Secretary of the Treasury, Steven Mnu...",https://www.coindesk.com/us-treasury-secretary...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T12:05:14Z,"The U.S. Secretary of the Treasury, Steven Mnu...",CoinDesk,"The U.S. Secretary of the Treasury, Steven Mnu..."
204,"{'id': None, 'name': 'CoinDesk'}",Omkar Godbole,Bull Grip Weakens As Bitcoin Price Breaks Belo...,Following the setting of a new record high of ...,https://www.coindesk.com/bull-grip-weakens-bit...,https://media.coindesk.com/uploads/2017/11/Cli...,2017-11-10T13:31:26Z,Bitcoin remains on the back foot ahead of the ...,CoinDesk,Bitcoin remains on the back foot ahead of the ...
...,...,...,...,...,...,...,...,...,...,...
743942,"{'id': None, 'name': 'CoinDesk'}",Jamie Crawley,Gemini to Cease Offering Crypto Services in Ne...,The exchange said it is taking the step due to...,https://www.coindesk.com/policy/2023/09/29/gem...,https://www.coindesk.com/resizer/5JOs1BQJ79kue...,2023-09-29T12:47:11Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Cryptocurrency exchange Gemini will stop offer...
743960,"{'id': None, 'name': 'CoinDesk'}","Lyllah Ledesma, Omkar Godbole",First Mover Americas: Circle Argues Stablecoin...,The latest price moves in bitcoin (BTC) and cr...,https://www.coindesk.com/markets/2023/09/29/fi...,https://www.coindesk.com/resizer/iDYQEx_XIxIfl...,2023-09-29T13:10:20Z,This article originally appeared in First Move...,CoinDesk,This article originally appeared in First Move...
744030,"{'id': None, 'name': 'CoinDesk'}",Danny Chong,Where Liquid Staking Meets Tokenization,"The crypto industry is changing rapidly, and l...",https://www.coindesk.com/consensus-magazine/20...,https://www.coindesk.com/resizer/Y0wciIP16Z0Yw...,2023-09-29T15:00:21Z,Liquid staking has the potential to be the bui...,CoinDesk,Liquid staking has the potential to be the bui...
744047,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Valkyrie Backs Off Ether Futures Purchases Unt...,The asset manager Thursday said it had already...,https://www.coindesk.com/markets/2023/09/29/va...,https://www.coindesk.com/resizer/psYGVjbtB9QL0...,2023-09-29T15:24:23Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Valkyrie Friday morning said it will not purch...


In [18]:
Seeking_Alpha.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\Seeking_Alpha_6.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content